In [ ]:
#necessary libraries
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
data = pd.read_csv('/content/full_hin.csv')

In [ ]:
data_test = pd.read_csv('/content/trac2_hin_dev.csv')

In [ ]:
data_test.head()

ID  ... Sub-task B
0    C38.9  ...       NGEN
1  C4.1510  ...       NGEN
2   C19.95  ...       NGEN
3   C4.281  ...       NGEN
4   C4.216  ...       NGEN

[5 rows x 4 columns]

In [ ]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
data

Text  label
0     Bollywood film dekhne ke samay logic ghar mein...      0
1                                      Chutiya movie...      0
2          Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने      0
3     @Pankaj Chauhan abey tum jse aadmiyo ko ye lgt...      0
4                    bro house of card ka review karona      0
...                                                 ...    ...
7547  अपने बाल बौद्ध दिवस में कुतिया बालिसों को भाग्...      1
7548  आप कौन हैं, अधिकारी चिलिस एनकी?आने वाले ग्रामी...      1
7549                                    असत्य और व्यर्थ      1
7550                                 शालिस घर शाली फकनी      1
7551                          यह बस हमारे ध्यान में आया      1

[7552 rows x 2 columns]

In [ ]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [ ]:
data

Text  label
0     Bollywood film dekhne ke samay logic ghar mein...      0
1                                      Chutiya movie...      0
2          Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने      0
3     @Pankaj Chauhan abey tum jse aadmiyo ko ye lgt...      0
4                    bro house of card ka review karona      0
...                                                 ...    ...
7547  अपने बाल बौद्ध दिवस में कुतिया बालिसों को भाग्...      1
7548  आप कौन हैं, अधिकारी चिलिस एनकी?आने वाले ग्रामी...      1
7549                                    असत्य और व्यर्थ      1
7550                                 शालिस घर शाली फकनी      1
7551                          यह बस हमारे ध्यान में आया      1

[7552 rows x 2 columns]

In [ ]:
NAG = data_test[(data_test['Sub-task A']=='NAG')]
OAG = data_test[(data_test['Sub-task A']=='CAG')]
CAG = data_test[(data_test['Sub-task A']=='OAG')]

In [ ]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG.head()

Text  Sub-task A
1   Bhaiya shaadi mein zaroor aana movie ka plot j...           0
2   Section 375 hai kya??? .... Ye to batate kam s...           0
4              Maine itni kam dislike kbhi nhii dekhi           0
7   जिश देश मे super30 जैसी फ़िल्म फ्लॉप ओर कबीर सि...           0
11            Bhai movie review 5 se 7 mins Bano yaar           0

In [ ]:
data_test = pd.concat([NAG,
                  OAG,
                  CAG
                 ],axis=0)

In [ ]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
#from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
#import transformers

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=f099083b066ffea6888e54b55270d3aefd65fa9a008b57542cafebfd1dea7411
  Stored in directory: /tmp/pip-ephem-wheel-cache-1xbv5185/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [30]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [31]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [32]:
def get_vec(x):
  doc = nlp(str(x))
  vec = doc.vector
  return vec

In [33]:
data['vec'] = data['Text'].apply(lambda x: get_vec(x))

In [34]:
data_test['vec'] = data_test['Text'].apply(lambda x: get_vec(x))

In [35]:
X = data['vec'].to_numpy()
X = X.reshape(-1, 1)

In [36]:
P = data_test['vec'].to_numpy()
P = P.reshape(-1, 1)

In [37]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [38]:
P = np.concatenate(np.concatenate(P, axis = 0), axis = 0).reshape(-1, 300)

In [39]:
y = data['label']

In [40]:
Q = data_test['Sub-task A']

In [41]:
X_train = X
X_test = P
y_train = y 
y_test = Q



In [42]:
clf = LogisticRegression(solver='liblinear')

In [43]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred = clf.predict(X_test)

In [45]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.64      0.70       578
           1       0.31      0.49      0.38       211
           2       0.52      0.47      0.49       208

    accuracy                           0.57       997
   macro avg       0.53      0.53      0.52       997
weighted avg       0.62      0.57      0.59       997

